<a href="https://www.kaggle.com/code/averma111/torch-optuna?scriptVersionId=129738864" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [31]:
!pip install optuna==3.1.0

In [32]:
import os, sys
import argparse
import numpy as np
import pandas as pd

import json
import optuna
import torch
from optuna.trial import TrialState
from torch.utils.data import DataLoader,Dataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection  import train_test_split
import warnings
warnings.filterwarnings('ignore')

torch.manual_seed(42)

In [33]:
class BostonData(Dataset):
    def __init__(self,X,y,scale_data=True):
    # Apply scaling if necessary
      if scale_data:
            X = StandardScaler().fit_transform(X)
            self.X = torch.from_numpy(X)
            self.y = torch.from_numpy(y)
            
    def __getitem__(self,item):
        return self.X[item], self.y[item]
    
    def __len__(self):
        return len(self.X)

In [34]:
def MLP(trial,in_features,n_layers,dropout,n_output):
    
    layers = []
    fc_layer = in_features
    
    for i in range(n_layers):
        
        out_features = trial.suggest_int("n_units_l{}".format(i),2,in_features)
        layers.append(torch.nn.Linear(in_features, out_features))
        layers.append(torch.nn.LeakyReLU())
        in_features = out_features

    layers.append(torch.nn.Linear(in_features, fc_layer)) 
    layers.append(torch.nn.LeakyReLU())

    layers.append(torch.nn.Dropout(dropout))
    layers.append(torch.nn.Linear(fc_layer,n_output)) 
    
    return torch.nn.Sequential(*layers)

In [35]:
def train_net(trial, params):
    # Load Boston dataset
    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    y = raw_df.values[1::2, 2]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    # Prepare Boston dataset
    trainloader = torch.utils.data.DataLoader(BostonData(X_train, y_train), batch_size=10, shuffle=True, num_workers=1)
    testloader = torch.utils.data.DataLoader(BostonData(X_test, y_test), batch_size=1, shuffle=True, num_workers=1)
    
    # Initialize the MLP
    net = MLP(trial,
            in_features= X.shape[1],
            n_layers=params['n_layers'] ,
            dropout=params['dropout'],
            n_output= 1)
    
  # Define the loss function and optimizer
    loss_function = torch.nn.MSELoss()
    optimizer = getattr(torch.optim, params['optimizer'])(net.parameters(), lr= params['learning_rate'], weight_decay=params['weight_decay'])
  
    # Run the training loop
    for epoch in range(0, 5): # 5 epochs at maximum
        
        total_test_loss = []
        # Print epoch
        #print(f'Starting epoch {epoch+1}')
        
        # Set current loss value
        current_loss = 0.0
        
        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):
        
            # Get and prepare inputs
            inputs, targets = data
            inputs, targets = inputs.float(), targets.float()
            targets = targets.reshape((targets.shape[0], 1))
            
            # Zero the gradients
            optimizer.zero_grad()
            # Perform forward pass
            outputs = net(inputs)
            # Compute loss
            loss = loss_function(outputs, targets)
            # Perform backward pass
            loss.backward()
            # Perform optimization
            optimizer.step()
            
        net.eval()
        with torch.no_grad():
            for i, data in enumerate(testloader, 0):
                # Get and prepare inputs
                inputs, targets = data
                inputs, targets = inputs.float(), targets.float()
                targets = targets.reshape((targets.shape[0], 1))
                
                # # Perform forward pass
                test_outputs = net(inputs)
                test_loss = loss_function(test_outputs, targets)
                total_test_loss.append(test_loss.item())
        

    # Process is complete.
    return total_test_loss

In [36]:
def objective(trial):
    """
    Objective function to run bayesian hyperparameter tuning.

    :param trial: optuna study
    :param checkpoint_dir: checkpoint dir args
    :param cfg: config file
    :return: mean RMSE test loss
    """ 
    # For TPESampler 
    params = {
              'learning_rate': trial.suggest_loguniform('learning_rate', 1e-6, 1e-2), 
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              'weight_decay': trial.suggest_loguniform('weight_decay', 1e-4, 1e-2),
              "n_layers" : trial.suggest_int("n_layers", 1, 4),
              "dropout" : trial.suggest_float('dropout',0.1,0.5,step = 0.1)
              }
    test_loss   = train_net(trial, params) 
    return np.mean(test_loss)

In [37]:
study = optuna.create_study(direction ="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials= 100,timeout=600)

    
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))
trial = study.best_trial
print("Best trial:", trial)
print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2023-05-16 06:14:38,366] A new study created in memory with name: no-name-f8d14231-02d2-4057-84c6-37aa29c45c74
[I 2023-05-16 06:14:42,049] Trial 0 finished with value: 578.3277875749689 and parameters: {'learning_rate': 2.2139671092870446e-06, 'optimizer': 'Adam', 'weight_decay': 0.0012431054498090546, 'n_layers': 1, 'dropout': 0.2, 'n_units_l0': 4}. Best is trial 0 with value: 578.3277875749689.
[I 2023-05-16 06:14:45,669] Trial 1 finished with value: 492.5045600564856 and parameters: {'learning_rate': 9.271350233709358e-05, 'optimizer': 'SGD', 'weight_decay': 0.0001707485914295625, 'n_layers': 2, 'dropout': 0.5, 'n_units_l0': 2, 'n_units_l1': 2}. Best is trial 1 with value: 492.5045600564856.
[I 2023-05-16 06:14:49,701] Trial 2 finished with value: 94.99858632836167 and parameters: {'learning_rate': 0.004342759007161644, 'optimizer': 'Adam', 'weight_decay': 0.00166578454335367, 'n_layers': 4, 'dropout': 0.5, 'n_units_l0': 9, 'n_units_l1': 7, 'n_units_l2': 5, 'n_units_l3': 3}. Best

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  0
  Number of complete trials:  100
Best trial: FrozenTrial(number=76, state=TrialState.COMPLETE, values=[18.76397387967623], datetime_start=datetime.datetime(2023, 5, 16, 6, 19, 34, 853923), datetime_complete=datetime.datetime(2023, 5, 16, 6, 19, 38, 611371), params={'learning_rate': 0.0020535183271016486, 'optimizer': 'SGD', 'weight_decay': 0.008684980402144278, 'n_layers': 2, 'dropout': 0.2, 'n_units_l0': 12, 'n_units_l1': 8}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.01, log=True, low=1e-06, step=None), 'optimizer': CategoricalDistribution(choices=('Adam', 'RMSprop', 'SGD')), 'weight_decay': FloatDistribution(high=0.01, log=True, low=0.0001, step=None), 'n_layers': IntDistribution(high=4, log=False, low=1, step=1), 'dropout': FloatDistribution(high=0.5, log=False, low=0.1, step=0.1), 'n_units_l0': IntDistribution(high=13, log=False,

In [38]:
optuna.visualization.plot_optimization_history(study)

In [39]:
optuna.visualization.plot_param_importances(study)

In [40]:
optuna.visualization.plot_slice(study)

In [41]:
optuna.visualization.plot_parallel_coordinate(study)